In [ ]:
#Reference: https://towardsdatascience.com/clustering-sentence-embeddings-to-identify-intents-in-short-text-48d22d3bf02e

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir("/content/drive/MyDrive/RIR 2022 Synthetic Data") #changing working directory to google drive

In [ ]:
! pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 106.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 129.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=f3b0529517df0127b0e3e32e016b7fdf6973281c3553d60e8fd09ecb253c0e75
  Stored in directory: /root/.cache/pip/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e8e11f
Successfully built sentence_transformers


In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from sentence_transformers import SentenceTransformer

In [ ]:
df=pd.read_csv("clean_tweets.csv")
df.head(5)

,Unnamed: 0,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category,cleaned_content
0,0,9.060000e+17,10_GOP,"""We have a sitting Democrat US Senator on tria...",Unknown,English,10/1/2017 19:58,10/1/2017 19:59,1052,9636,253,NaN,Right,0,0,RightTroll,sitting democrat us senator trial corruption b...
1,1,9.060000e+17,10_GOP,Marshawn Lynch arrives to game in anti-Trump s...,Unknown,English,10/1/2017 22:43,10/1/2017 22:43,1054,9637,254,NaN,Right,0,0,RightTroll,marshawn lynch arrives game anti trump shirt j...
2,2,9.060000e+17,10_GOP,Daughter of fallen Navy Sailor delivers powerf...,Unknown,English,10/1/2017 22:50,10/1/2017 22:51,1054,9637,255,RETWEET,Right,0,1,RightTroll,daughter fallen navy sailor delivers powerful ...
3,3,9.060000e+17,10_GOP,JUST IN: President Trump dedicates Presidents ...,Unknown,English,10/1/2017 23:52,10/1/2017 23:52,1062,9642,256,NaN,Right,0,0,RightTroll,president trump dedicates presidents cup golf ...
4,4,9.060000e+17,10_GOP,"19,000 RESPECTING our National Anthem! #StandF...",Unknown,English,10/1/2017 2:13,10/1/2017 2:13,1050,9645,246,RETWEET,Right,0,1,RightTroll,19 000 respecting national anthem


In [ ]:
df["cleaned_content"]=df["cleaned_content"].astype(str)

In [ ]:
df_sample = df.sample(frac=0.005)

In [ ]:
all_intents = list(df_sample["cleaned_content"])

In [ ]:
# define the document embedding models to use for comparison
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4"
model_use = hub.load(module_url)
model_st1 = SentenceTransformer('all-mpnet-base-v2')
model_st2 = SentenceTransformer('all-MiniLM-L6-v2')
model_st3 = SentenceTransformer('paraphrase-mpnet-base-v2')


In [ ]:
def embed(model, model_type, sentences):
  # This is a wrapper function for generating embedding
  if model_type == 'use':
    embeddings = model(sentences)
  elif model_type == 'sentence transformer':
    embeddings = model.encode(sentences)
    return embeddings

In [ ]:
# generate embeddings for each model
#embeddings_use = embed(model_use, 'use', all_intents)
embeddings_st1 = embed(model_st1, 'sentence transformer', all_intents) # type is numpy.ndarray
#embeddings_st2 = embed(model_st2, 'sentence transformer', all_intents)
#embeddings_st3 = embed(model_st3, 'sentence transformer', all_intents)

In [ ]:
#embeddings = [embeddings_st1, embeddings_st2, embeddings_st3]
embeddings = [embeddings_st1]
for embedding in embeddings:
  print(embedding.shape)

(14867, 768)


In [ ]:
#! pip install umap
! pip install hdbscan

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 50.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for hdbscan: filename=hdbscan-0.8.29-cp39-cp39-linux_x86_64.whl size=3582074 sha256=1186c0cce052f2aec28ac3b25b42234a472424ac4f43e14b143063b1ad651587
  Stored in directory: /root/.cache/pip/wheels/05/6f/88/1a4c04276b98306f00217a1e300e6ba0252c6aa4f7616067ae
Successfully built hdbscan


In [ ]:
# Generate HDBSCAN cluster object after reducing embedding dimensionality with UMAP

In [ ]:
#import umap
import hdbscan
import numpy as np

In [ ]:
#! pip uninstall umap
! pip install umap-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.2/88.2 KB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=3b8a4ad0e494d2b8e5d5c7f37b6f2490d6b8bc2fa0015bc37eeef56be00b6350
  Stored in directory: /root/.cache/pip/wheels/f4/3e/1c/596d0a463d17475af648688443fa4846fef624d1390339e7e9
  Created wheel for pynndescent: filename=pynndescent-0.5.8-py3-none-any.whl size=55513 sha256=6c61507472d012f51dc595ffd00279c89e5f8f40ae446ac5e9ef739773a1a284
  Stored in directory: /root/.cache/pip/wheels/b9/89/cc/59ab91ef5b21dc2ab3635528d7d227f49dfc9169905dcb959d
Successfully built umap-learn pynndescent


In [ ]:
import umap.umap_ as umap

In [ ]:
def generate_clusters(message_embeddings,
                      n_neighbors,
                      n_components,
                      min_cluster_size,
                      random_state = None):
    umap_embeddings = (umap.UMAP(n_neighbors=n_neighbors,
                                n_components=n_components,
                                metric='cosine',
                                random_state=random_state)
                            .fit_transform(message_embeddings))

    clusters = hdbscan.HDBSCAN(min_cluster_size = min_cluster_size,
                               metric='euclidean',
                               cluster_selection_method='eom').fit(umap_embeddings)

    return clusters

In [ ]:
# Returns the label count and cost of a given cluster supplied from running hdbscan

In [ ]:
def score_clusters(clusters, prob_threshold = 0.05):
    cluster_labels = clusters.labels_
    label_count = len(np.unique(cluster_labels))
    total_num = len(clusters.labels_)
    cost = (np.count_nonzero(clusters.probabilities_ < prob_threshold)/total_num)

    return label_count, cost

In [ ]:
# prompt: # prompt: Can you look at the entire code and tell me what is this code doing

This code performs clustering on sentence embeddings derived from a sample of cleaned tweets. Let's break down the code step by step:

1. **Setup and Data Loading:**
   - Imports necessary libraries (pandas, TensorFlow, Sentence Transformers, HDBSCAN, UMAP, etc.).
   - Mounts Google Drive and changes the working directory to a specific folder within the drive.
   - Installs the `sentence_transformers` library.
   - Reads a CSV file named "clean_tweets.csv" into a pandas DataFrame.
   - Takes a small sample (0.5%) of the tweets for processing.
   - Extracts the cleaned tweet content into a list called `all_intents`.


2. **Sentence Embedding Generation:**
   - Defines two sentence embedding models:  `all-mpnet-base-v2` from Sentence Transformers and one from TF Hub's Universal Sentence Encoder.  Other models were included but commented out.
   - Creates an `embed` function to generate embeddings using either model.
   - Generates embeddings for the sampled tweets using the selected Sentence Transformer model.


3. **Dimensionality Reduction and Clustering:**
   - Installs `hdbscan` and `umap-learn`.
   - Defines a `generate_clusters` function that performs the following:
     - Reduces the dimensionality of the sentence embeddings using UMAP, with adjustable parameters for `n_neighbors`, `n_components`, and `min_cluster_size`.  It uses cosine similarity for distance metrics.
     - Clusters the reduced embeddings using HDBSCAN, with parameters for `min_cluster_size` and `cluster_selection_method`.


4. **Cluster Scoring:**
   - Defines a `score_clusters` function. This function analyzes the generated clusters, counting unique labels and calculating a cost based on the probability threshold. This helps in evaluating the quality of clustering.


**In summary:** The code takes a sample of tweet text, generates sentence embeddings using a pre-trained model, reduces the dimensionality of those embeddings using UMAP, and clusters the tweets based on their embedding similarity using HDBSCAN. It then assesses the quality of the clusters based on the number of clusters and a probabilistic measure.  The code is structured to allow exploration of different models (though most are commented out in the provided code),  UMAP parameters and HDBSCAN parameters for finding optimal clustering results.


Random Hyperparameter Search

In [ ]:
from tqdm import trange

In [ ]:
import random

In [ ]:
def random_search(embeddings, space, num_evals):
    """
    Randomly search hyperparameter space and limited number of times
    and return a summary of the results
    """

    results = []

    for i in trange(num_evals):
        n_neighbors = random.choice(space['n_neighbors'])
        n_components = random.choice(space['n_components'])
        min_cluster_size = random.choice(space['min_cluster_size'])

        clusters = generate_clusters(embeddings,
                                     n_neighbors = n_neighbors,
                                     n_components = n_components,
                                     min_cluster_size = min_cluster_size,
                                     random_state = 42)

        label_count, cost = score_clusters(clusters, prob_threshold = 0.05)

        results.append([i, n_neighbors, n_components, min_cluster_size,
                        label_count, cost])

    result_df = pd.DataFrame(results, columns=['run_id', 'n_neighbors', 'n_components',
                                               'min_cluster_size', 'label_count', 'cost'])

    return result_df.sort_values(by='cost')

In [ ]:
space = {"n_neighbors": range(12,16), "n_components": range(3,7), "min_cluster_size": range(2,16), "random_state": 42}
random_use = random_search(embeddings_st1, space, 100)

100%|██████████| 100/100 [18:34<00:00, 11.15s/it]


In [ ]:
type(random_use)

pandas.core.frame.DataFrame

In [ ]:
len(random_use)

100

In [ ]:
random_use.head(50)

,run_id,n_neighbors,n_components,min_cluster_size,label_count,cost
55,55,13,5,15,22,0.005112
2,2,12,4,9,29,0.005179
50,50,12,4,9,29,0.005179
66,66,13,5,12,25,0.006188
53,53,13,5,12,25,0.006188
31,31,12,4,12,27,0.006457
45,45,14,4,13,25,0.007264
82,82,12,4,10,29,0.007264
5,5,13,4,7,38,0.007399
77,77,15,3,10,29,0.007870


cost is (cost *100) percentage of data labeled as outliers or low confidence. You want it low as possible.

In [ ]:
# of neighbors 14, # of components 6, min cluster size 13

In [ ]:
cluster = generate_clusters(embeddings_st1, n_neighbors=14, n_components=6, min_cluster_size=13)

In [ ]:
type(cluster)

hdbscan.hdbscan_.HDBSCAN

In [ ]:
len(cluster.labels_) # same as the tweet length

14867

In [ ]:
labels = cluster.labels_

In [ ]:
type(labels)

numpy.ndarray

In [ ]:
df_sample["clusters"]= labels

In [ ]:
df_sample.head(5)

,Unnamed: 0,external_author_id,author,content,region,language,publish_date,harvested_date,following,followers,updates,post_type,account_type,new_june_2018,retweet,account_category,cleaned_content,clusters
2466547,2466547,2.611151e+09,SEATTLE_POST,Five quick hits from Seahawks’ Tuesday OTA #news,United States,English,6/2/2015 22:36,6/2/2015 22:36,8250,11679,5355,NaN,local,0,0,NewsFeed,five quick hits seahawks tuesday ota,43
1779994,1779994,2.259655e+09,MILA_NORD,«Исламское государство» взяло на себя ответств...,United States,Russian,12/12/2015 20:53,12/12/2015 20:53,1805,1348,3534,RETWEET,Russian,1,1,NonEnglish,nan,0
610458,610458,1.683408e+09,CHRIXCHASE,#2016In4Words can we cancel 2016?,United States,English,12/28/2016 16:49,12/28/2016 16:50,3319,3003,3562,RETWEET,Hashtager,1,1,HashtagGamer,cancel 2016,-1
1144816,1144816,1.674084e+09,GAB1ALDANA,He was the tour lighting director for Bone Thu...,United States,English,10/11/2016 7:48,10/11/2016 7:48,3344,3069,3504,NaN,Hashtager,0,0,HashtagGamer,tour lighting director bone thugs n harmony go...,68
2296340,2296340,2.570575e+09,RIAFANRU,Глава Ростуризма рассказал о причинах банкротс...,Belarus,Russian,8/25/2017 17:15,8/25/2017 17:15,6492,12905,91386,NaN,Russian,0,0,NonEnglish,ted travel,61


In [ ]:
! pip install spaCy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import spacy

/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
nlp = spacy.load("en_core_web_sm")

In [ ]:
import collections

In [ ]:
from collections import Counter

In [ ]:
verbs = []
dobjs = []
nouns = []
adjs = []
verb = ''
dobj = ''
noun1 = ''
noun2 = ''

In [ ]:
text = "I am Swaptik"

In [ ]:
doc = nlp(text)

In [ ]:
type(doc)

spacy.tokens.doc.Doc

In [ ]:
def extract_labels(category_docs):
    """
    Extract labels from documents in the same cluster by concatenating
    most common verbs, ojects, and nouns
    """

    verbs = []
    dobjs = []
    nouns = []
    adjs = []

    verb = ''
    dobj = ''
    noun1 = ''
    noun2 = ''

    # for each document, append verbs, dobs, nouns, and adjectives to
    # running lists for whole cluster
    for i in range(len(category_docs)): # length of all tweets
        doc = nlp(category_docs[i]) # each tweets in the df
        for token in doc:
            if token.is_stop==False:
                if token.dep_ == 'ROOT':
                    verbs.append(token.text.lower())

                elif token.dep_=='dobj':
                    dobjs.append(token.lemma_.lower())

                elif token.pos_=='NOUN':
                    nouns.append(token.lemma_.lower())

                elif token.pos_=='ADJ':
                    adjs.append(token.lemma_.lower())

    # take most common words of each form
    if len(verbs) > 0:
        verb = most_common(verbs, 1)[0][0]

    if len(dobjs) > 0:
        dobj = most_common(dobjs, 1)[0][0]

    if len(nouns) > 0:
        noun1 = most_common(nouns, 1)[0][0]

    if len(set(nouns)) > 1:
        noun2 = most_common(nouns, 2)[1][0]

    # concatenate the most common verb-dobj-noun1-noun2 (if they exist)
    label_words = [verb, dobj]

    for word in [noun1, noun2]:
        if word not in label_words:
            label_words.append(word)

    if '' in label_words:
        label_words.remove('')

    label = '_'.join(label_words)

    return label